# Loading the Model for Inference

In [6]:
import pandas as pd
df_test = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/Assignment_for_interview/Dataset/test.csv")
df_test.head()

,Class Index,Title,Description
0,3,Fears for T N pension after talks,Unions representing workers at Turner Newall...
1,4,The Race is On: Second Private Team Sets Launc...,"SPACE.com - TORONTO, Canada -- A second\team o..."
2,4,Ky. Company Wins Grant to Study Peptides (AP),AP - A company founded by a chemistry research...
3,4,Prediction Unit Helps Forecast Wildfires (AP),AP - It's barely dawn when Mike Fitzpatrick st...
4,4,Calif. Aims to Limit Farm-Related Smog (AP),AP - Southern California's smog-fighting agenc...


In [2]:
import torch
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [3]:
save_directory = "/content/drive/MyDrive/model"
from transformers import AutoModelForSequenceClassification, AutoTokenizer
model = AutoModelForSequenceClassification.from_pretrained(save_directory).to(device)
tokenizer = AutoTokenizer.from_pretrained(save_directory)


In [7]:
df_test['Class Index'].replace([1, 2, 3, 4],['World', 'Sports', 'Business', 'Sci/Tech'], inplace=True)

In [8]:
df_test['label'] = df_test['Class Index'].astype('category').cat.codes

In [9]:
df_test['text'] = df_test['Title'] + ' ' + df_test['Description']

In [10]:
test_df = df_test[['text','label']]

In [14]:

input_texts = test_df['text'].to_list()
predicted_classes = []

for input_text in input_texts:
    inputs = tokenizer(input_text, return_tensors="pt", padding=True, truncation=True).to(device)
    logits = model(**inputs).logits
    predicted_class = logits.argmax().item()
    predicted_classes.append(predicted_class)




In [15]:
from sklearn.metrics import classification_report
report = classification_report(test_df['label'].to_list(), predicted_classes, target_names=['World', 'Sports', 'Business', 'Sci/Tech'])
print(report)


              precision    recall  f1-score   support

       World       0.93      0.91      0.92      1900
      Sports       0.91      0.93      0.92      1900
    Business       0.99      0.99      0.99      1900
    Sci/Tech       0.96      0.95      0.96      1900

    accuracy                           0.94      7600
   macro avg       0.94      0.94      0.94      7600
weighted avg       0.94      0.94      0.94      7600

